In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Function to extract chords from JSON file
def extract_chords_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        songs_data = data.get('songs', [])
        chord_sequences = {}
        for song in songs_data:
            title = song.get('title', 'Unknown')
            measures = song.get('music', {}).get('measures', [])
            chords = [chord for measure in measures for chord in measure]
            chord_sequences[title] = chords
        return chord_sequences

# Path to your JSON file in Google Drive
json_file_path = '/content/drive/My Drive/Colab Notebooks/MARG/irealPro/playlist.json'

# Extract chord sequences
chord_sequences = extract_chords_from_json(json_file_path)

# Prepare data for training
all_chords = [chord for chords in chord_sequences.values() for chord in chords]
label_encoder = LabelEncoder()
encoded_chords = label_encoder.fit_transform(all_chords)

sequence_length = 4
X, y = [], []

for chords in chord_sequences.values():
    song_encoded_chords = label_encoder.transform(chords)
    for i in range(len(song_encoded_chords) - sequence_length):
        X.append(song_encoded_chords[i : i + sequence_length])
        y.append(song_encoded_chords[i + sequence_length])

X = torch.LongTensor(X)
y = torch.LongTensor(y)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# LSTMChordGenerator class remains unchanged
class LSTMChordGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMChordGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=2)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, cell) = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# Model, loss function, and optimizer setup
vocab_size = len(label_encoder.classes_)
embedding_dim = 50
hidden_dim = 100

model = LSTMChordGenerator(vocab_size, embedding_dim, hidden_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(100):
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

# Generate Sequence Function
def generate_sequence(seed_text, n_words, model, label_encoder):
    model.eval()
    in_text = label_encoder.transform(seed_text).tolist()
    result = []
    for _ in range(n_words):
        in_tensor = torch.LongTensor([in_text]).to(device)
        with torch.no_grad():
            output = model(in_tensor)
        next_chord_index = torch.argmax(output, dim=1).item()
        next_chord = label_encoder.inverse_transform([next_chord_index])[0]
        result.append(next_chord)
        in_text.append(next_chord_index)
        in_text.pop(0)
    return ' '.join(result)

# Example usage
seed_chords = ['D-7', 'G7', 'C^7', 'A7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(generated_sequence)


Epoch 0, Loss: 3.3653242588043213
Epoch 1, Loss: 3.6338119506835938
Epoch 2, Loss: 1.387737512588501
Epoch 3, Loss: 4.170721054077148
Epoch 4, Loss: 1.2542552947998047
Epoch 5, Loss: 1.872710943222046
Epoch 6, Loss: 1.797762155532837
Epoch 7, Loss: 2.6316869258880615
Epoch 8, Loss: 0.9039561152458191
Epoch 9, Loss: 2.057141065597534
Epoch 10, Loss: 0.2810600996017456
Epoch 11, Loss: 1.5471161603927612
Epoch 12, Loss: 1.7013574838638306
Epoch 13, Loss: 0.9674204587936401
Epoch 14, Loss: 1.897194504737854
Epoch 15, Loss: 1.6746324300765991
Epoch 16, Loss: 1.9725176095962524
Epoch 17, Loss: 0.9532656073570251
Epoch 18, Loss: 1.362269639968872
Epoch 19, Loss: 0.9669406414031982
Epoch 20, Loss: 0.6622140407562256
Epoch 21, Loss: 0.947912871837616
Epoch 22, Loss: 1.1293312311172485
Epoch 23, Loss: 0.592888593673706
Epoch 24, Loss: 1.189862608909607
Epoch 25, Loss: 1.2293034791946411
Epoch 26, Loss: 1.1263256072998047
Epoch 27, Loss: 1.1398812532424927
Epoch 28, Loss: 1.0504059791564941
Epoch

In [ ]:
# Example usage
seed_chords = ['D-7', 'G7', 'C^7', 'A7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['D-7', 'G7', 'G-7', 'C7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['F^7', 'Eh7', 'A7', 'D-7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['D-6', 'D-6', 'D-6', 'D-6']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

['D-7', 'G7', 'C^7', 'A7'] D-7 G7 E-7 A7 D-7 G7 C6 D-7 G7 C6 C6 G7/D C G7 C6 G7/D C D-7 G7sus C6
['D-7', 'G7', 'G-7', 'C7'] F^7 Bb7 A-7 Abo7 G-7 C7 F^7 Bh7 E7b9 A-7 A-7 G7sus G7 C^7 Bh E7 A-7 Bh7 Bh7 E7b9
['F^7', 'Eh7', 'A7', 'D-7'] Eh7 A7 D-7 D7 G7 C7 Eh7 A7 D-7 Eh7 A7 D-7 D7 G7 C7 Eh7 A7 D-7 Eh7 A7
['D-6', 'D-6', 'D-6', 'D-6'] D6 D6 D6 D6 D-6 D-6 D-6 D-6 D6 D6 D6 D6 D-6 D-6 D-6 D-6 D6 D6 D6 D6
